In [96]:
import requests
import pandas as pd 
import time
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium_stealth import stealth
from webdriver_manager.chrome import ChromeDriverManager
from webdriver_manager.core.os_manager import ChromeType 

In [9]:
def get_wb_products(query="электростимулятор", pages=3):
    all_products = []
    
    for page in range(1, pages + 1):
        url = "https://search.wb.ru/exactmatch/ru/common/v4/search"
        params = {
            "query": query,
            "resultset": "catalog",
            "limit": 100,
            "page": page,
            "appType": 1,
            "curr": "rub",
            "dest": -1257786  # Регион доставки: Москва
        }
        
        response = requests.get(url, params=params)
        if response.status_code != 200:
            print(f"Ошибка на странице {page}")
            continue
            
        data = response.json()
        products = data.get("data", {}).get("products", [])
        
        for product in products:
            all_products.append({
                "id": product["id"],
                "name": product["name"],
                "price": product["salePriceU"] / 100,
                "rating": product.get("reviewRating", 0),
                "feedbacks": product.get("feedbacks", 0),
                "brand": product["brand"]
            })
            
    return pd.DataFrame(all_products)

# Собираем 300 товаров (3 страницы)
df = get_wb_products(pages=3)
df.to_csv("wb_products.csv", index=False)
print(f"Собрано {len(df)} товаров")

Собрано 300 товаров


In [ ]:
# Инициализация драйвера один раз (вне функции)
def init_driver_Chrome():
    chrome_options = Options()
    # chrome_options.add_argument("--headless=new")
    chrome_options.add_argument("--disable-gpu")
    chrome_options.add_argument("--no-sandbox")
    chrome_options.add_argument("--disable-dev-shm-usage")
    
    # Для автоматического определения архитектуры M1
    service = Service(ChromeDriverManager().install())
    driver = webdriver.Chrome(service=service, options=chrome_options)
    stealth(driver, 
            platform="macOS",
            languages=["en-US", "en"],
            webgl_vendor="Intel Inc.")
    
    return driver

In [229]:
from selenium import webdriver
from selenium.webdriver.firefox.service import Service
from selenium.webdriver.firefox.options import Options
from webdriver_manager.firefox import GeckoDriverManager
import random

def init_driver_firefox(headless = False):
    firefox_options = Options()

    if headless:
        firefox_options.add_argument("--headless")
        firefox_options.set_preference("layout.css.devPixelsPerPx", "1")  # Фиксируем масштаб
        
    # Настройки для ускорения и stealth
    firefox_options.set_preference("dom.webdriver.enabled", False)
    firefox_options.set_preference("useAutomationExtension", False)
    firefox_options.set_preference("browser.cache.disk.enable", True)
    firefox_options.set_preference("browser.cache.memory.enable", True)
    firefox_options.set_preference("browser.cache.offline.enable", True)
    firefox_options.set_preference("network.http.use-cache", True)
    firefox_options.set_preference("permissions.default.image", 2)  # Блокировка изображений
    
    # Случайный User-Agent из реальных браузеров
    user_agents = [
        "Mozilla/5.0 (Macintosh; Intel Mac OS X 10.15; rv:109.0) Gecko/20100101 Firefox/115.0",
        "Mozilla/5.0 (Macintosh; Intel Mac OS X 10.15; rv:122.0) Gecko/20100101 Firefox/122.0",
        "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/16.6 Safari/605.1.15"
    ]
    firefox_options.set_preference("general.useragent.override", random.choice(user_agents))
    
    # Дополнительные настройки для незаметности
    firefox_options.set_preference("privacy.resistFingerprinting", True)
    firefox_options.set_preference("privacy.trackingprotection.enabled", True)
    firefox_options.set_preference("dom.event.clipboardevents.enabled", False)
    firefox_options.set_preference("media.volume_scale", "0.0")
    
    # Настройки для производительности
    firefox_options.set_preference("gfx.webrender.all", True)
    firefox_options.set_preference("layers.acceleration.force-enabled", True)
    
    # Создаем сервис и драйвер
    service = Service(GeckoDriverManager().install())
    driver = webdriver.Firefox(service=service, options=firefox_options)
    
    # Устанавливаем размер окна как у реального пользователя
    driver.set_window_size(random.randint(1200, 1400), random.randint(800, 1000))
    
    # Эмулируем человеческое поведение
    driver.execute_script("Object.defineProperty(navigator, 'webdriver', {get: () => undefined})")
    driver.execute_script("window.chrome = undefined;")
    
    return driver

# Пример использования
# driver = init_driver_firefox()
# driver.get("https://www.wildberries.ru")
# # print(driver.title)
# driver.quit()

In [239]:
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

# Передаём драйвер как аргумент
def get_product_details(driver, product_id):
    driver.get(f"https://www.wildberries.ru/catalog/{product_id}/detail.aspx")
    details = {
        "description": "",
        "specifications": {}  # Словарь для всех характеристик
    }
    
    # Умное ожидание вместо фиксированного sleep
    WebDriverWait(driver, 30).until(
        EC.presence_of_element_located((By.CLASS_NAME, "main__container"))
    )
    
    # Проверка и клик по кнопке подтверждения возраста
    try:
        button_confirm_age = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.XPATH, "/html/body/div[1]/div/div/button[1]"))
        )
        button_confirm_age.click()
        print("Подтверждение возраста выполнено")
    except Exception:
        print("Кнопка подтверждения возраста не найдена")
    
    # Прокрутка с ожиданием
    driver.execute_script("window.scrollBy(0, 800)")

    # Нажатие на характеристики с улучшенным ожиданием
    try:
        button = WebDriverWait(driver, 15).until(
            EC.element_to_be_clickable((By.CSS_SELECTOR, "button.product-page__btn-detail.hide-mobile.j-details-btn-desktop"))
        )
        driver.execute_script("arguments[0].click();", button)
        
        # Ожидание загрузки контента характеристик
        try:
            WebDriverWait(driver, 15).until(
                EC.visibility_of_element_located((By.CSS_SELECTOR, ".product-params, .option__text"))
            )
            print("Характеристики успешно открыты")
        except:
            print("Характеристики не найдены")
        
        # Сбор данных
        try:
            time.sleep(1)
            details["description"] = driver.find_element(By.CSS_SELECTOR, ".option__text").text
            print("Описание успешно записано")
        except Exception:
            print("Описание не найдено")
            
        # Парсинг характеристик
        try:
            # Находим все таблицы с характеристиками
            tables = driver.find_elements(By.CSS_SELECTOR, "table.product-params__table")
            for table in tables:
                # Извлекаем название группы характеристик
                try:
                    group_name = table.find_element(By.CSS_SELECTOR, "caption.product-params__caption").text

                    # Создаем подраздел для этой группы
                    details["specifications"][group_name] = {}
                    
                    # Обрабатываем строки таблицы
                    rows = table.find_elements(By.CSS_SELECTOR, "tr.product-params__row")
                    for row in rows:
                        try:
                            name = row.find_element(By.CSS_SELECTOR, "th.product-params__cell").text.strip()
                            value = row.find_element(By.CSS_SELECTOR, "td.product-params__cell").text.strip()
                            
                            # Сохраняем характеристику в соответствующей группе
                            details["specifications"][group_name][name] = value
                            
                            # Для совместимости сохраняем особые поля
                            if 'питани' in name.lower():
                                details["power_type"] = value
                            elif 'зон' in name.lower() or 'область' in name.lower():
                                details["zones"] = value
                            
                                
                        except Exception as e:
                            print(f"Ошибка обработки строки: {str(e)}")
                            continue
                except:
                    continue
                        
        except Exception as e:
            print(f"Ошибка парсинга характеристик: {str(e)}")
            
    except Exception as e:
        print(f"Ошибка открытия характеристик: {str(e)}")
    
    return details

In [244]:
from config import *

driver = init_driver_firefox(browser_headless)

In [246]:
product_id = df.iloc[0]["id"]
details = get_product_details(driver, product_id)

Кнопка подтверждения возраста не найдена
Характеристики успешно открыты
Описание успешно записано


In [247]:
print(details)

{'description': 'Электростимулятор чрескожный универсальный Нейродэнс ПКМ — это современный физиотерапевтический аппарат, который станет надежным помощником в восстановлении и поддержании здоровья. Он предназначен для применения в домашних условиях, предоставляя возможность получать эффективные физиотерапевтические процедуры без необходимости посещения медицинских учреждений.\n\nПрибор работает на основе технологии электростимуляции, что позволяет воздействовать на мышцы, улучшая кровообращение и облегчая болевые ощущения. Устройство может использоваться для различных целей: от снятия напряжения после трудного дня до реабилитации после травм. С ним вы сможете проводить высококачественные процедуры, направленные на восстановление функциональности мышц и суставов, а также на долголетие здоровья.\n\nУниверсальный электростимулятор обладает несколькими режимами работы, которые легко переключаются в зависимости от потребностей пользователя. Компактный размер и стильный дизайн устройства поз

In [243]:
driver.quit()  # Закрываем браузер только после всех запросов

Тестыыыы

In [8]:
product_id = df.iloc[0]["id"]
driver.get(f"https://www.wildberries.ru/catalog/{product_id}/detail.aspx")

In [10]:
# Нажатие кнопки 18+
button_confirm_age = driver.find_element(By.XPATH, "/html/body/div[1]/div/div/button[1]")
button_confirm_age.click()

In [13]:
driver.execute_script("window.scrollBy(0, 800)")

In [19]:
try:
    # Нажатие на Характеристики
    button_characteristics_and_description = driver.find_element(By.CSS_SELECTOR, "button.product-page__btn-detail.hide-mobile.j-details-btn-desktop")
    button_characteristics_and_description.click()
except Exception as e:
    print(f"Ошибка для {product_id}: {str(e)}")

# <button type="button" class="product-page__btn-detail hide-mobile j-details-btn-desktop" data-name-for-wba="Item_Description_Parameters_More" data-link="{on showDetail}text{:~t('productCard.charAndDescr')}class{merge: abCentralBlock toggle='product-page__btn-detail--ab'}" data-jsv="#2190^/2190^">Характеристики и описание</button>

In [35]:
driver.quit()  # Закрываем браузер только после всех запросов

In [75]:
def get_product_details_1(nm_id):
    url = "https://card.wb.ru/cards/v2/detail"
    params = {
        "appType": 1,
        "curr": "rub",
        "dest": -1257786,  # Регион доставки (Москва)
        "nm": nm_id        # Артикул товара
    }
    
    response = requests.get(url, params=params)
    print(response)
    # data = response.json()
    
    # Извлекаем характеристики
    # product = data["data"]["products"][0]
    return {
        # "Характеристики": [f"{opt['name']}: {opt['value']}" for opt in product["options"]],
        # "Описание": product.get("description", "")
    }

# Пример использования 
details = get_product_details_1(428282572)
print(details)

<Response [200]>
{}


{'description': 'Электростимулятор чрескожный универсальный Нейродэнс ПКМ — это современный физиотерапевтический аппарат, который станет надежным помощником в восстановлении и поддержании здоровья. Он предназначен для применения в домашних условиях, предоставляя возможность получать эффективные физиотерапевтические процедуры без необходимости посещения медицинских учреждений.\n\nПрибор работает на основе технологии электростимуляции, что позволяет воздействовать на мышцы, улучшая кровообращение и облегчая болевые ощущения. Устройство может использоваться для различных целей: от снятия напряжения после трудного дня до реабилитации после травм. С ним вы сможете проводить высококачественные процедуры, направленные на восстановление функциональности мышц и суставов, а также на долголетие здоровья.\n\nУниверсальный электростимулятор обладает несколькими режимами работы, которые легко переключаются в зависимости от потребностей пользователя. Компактный размер и стильный дизайн устройства позволяют его удобно использовать как в домашних условиях, так и брать с собой в поездки. Медицинский прибор оснащен простым и интуитивно понятным управлением, что делает его доступным для всех возрастных категорий.\n\nБлагодаря высококачественным материалам и современным технологиям, этот физиотерапевтический аппарат обеспечивает надежность и долгий срок службы. Нейродэнс ПКМ идеально подходит для тех, кто заботится о своем здоровье и стремится к активной жизни.\n\nС этим электростимулятором вы можете легко осуществлять заботу о своем организме, не выходя из дома. Он станет надежным партнёром в процессе восстановления и улучшения самочувствия.', 
'specifications': 
{'Общие характеристики': {'Гарантийный срок': '1 год'}, 
'Питание': {'Питание': 'от батареек'}, 
'Технические особенности': {'Количество режимов работы': '24'}, 
'Материалы': {'Материал корпуса': 'пластик'}, 
'Дополнительная информация': {'Вид воздействия': 'ток низкой частоты на определённые зоны тела', 
'Область применения': 'для всего тела', 'Противопоказания': 'индивидуальная непереносимость; наличие имплантированного кардиостимулятора', 'Комплектация': 'батарейки АА (2шт.); инструкция по эксплуатации; электростимулятор', 'Страна производства': 'Россия'}, 
'Габариты': {'Высота предмета': '4.5 см', 'Ширина предмета': '5.5 см', 'Длина предмета': '14.5 см', 'Вес товара без упаковки (г)': '350 г', 'Вес товара с упаковкой (г)': '425 г', 'Длина упаковки': '24 см', 'Высота упаковки': '9 см', 'Ширина упаковки': '17 см'}, 
'Документы': {'Регистрационное удостоверение': 'РЗН 2019/9330'}, 
'Без названия': {'': ''}}, 
'power_type': 'от батареек'}